In [6]:
%matplotlib inline
from shapely.geometry import Polygon, mapping
from shapely.affinity import rotate, scale
from collections import defaultdict

import numpy as np
import cv2
from matplotlib import pyplot as plt


import torch
from torch import nn
from torch.nn import functional as F

from shapely.strtree import STRtree
import time

In [7]:
def rotated_bbox_to_contours(cx, cy, w, h, a):
    """
    Return the four contours for a rotated bounding box
    :param cx: center x coordinate
    :param cy: center y coordinate
    :param w: width of the rectangle
    :param h: height of the rectangle
    :param a: the center angle of the rectangle as degrees
    :return: ((x1, y1), (x2, y2), (x3, y3), (x4, y4))
    """
    theta = a * np.pi / 180 
    dx = w/2
    dy = h/2
    dxcos = dx * np.cos(theta)
    dxsin = dx * np.sin(theta)
    dycos = dy * np.cos(theta)
    dysin = dy * np.sin(theta)
    return (
        np.asarray([cx, cy]) + np.asarray([-dxcos - -dysin, -dxsin + -dycos]),
        np.asarray([cx, cy]) + np.asarray([dxcos - -dysin,  dxsin + -dycos]),
        np.asarray([cx, cy]) + np.asarray([dxcos -  dysin,  dxsin +  dycos]),
        np.asarray([cx, cy]) + np.asarray([-dxcos -  dysin, -dxsin +  dycos])
    )

def create_polygon(cx, cy, w, h, a):
    return Polygon(rotated_bbox_to_contours(cx, cy, w, h, a))

def create_polygon_list(rotated_bboxes):
    return [create_polygon(box[0], box[1], box[3], box[2], box[4]) for box in rotated_bboxes]

## Testing Anchor Grid with Shapely Rtree

In [8]:
def one_hot_embedding(labels, num_classes):
    return torch.eye(num_classes, dtype=torch.double)[labels.data.cpu()]

class BCE_Loss(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes

    def forward(self, pred, targ):
        t = one_hot_embedding(targ, self.num_classes+1)
        t = t[:,:-1].contiguous()#.cpu()
        x = pred[:,:-1]
        w = self.get_weight(x,t)
        #print(type(w))
        
        
        return F.binary_cross_entropy_with_logits(x, t, w, size_average=False)/self.num_classes
    
    def get_weight(self,x,t): return None

class FocalLoss(BCE_Loss):
    def get_weight(self,x,t):
        alpha,gamma = 0.25,2.
        p = x.sigmoid()
        pt = p*t + (1-p)*(1-t)
        w = alpha*t + (1-alpha)*(1-t)
        return w * (1-pt).pow(gamma)   
    
    



In [12]:
class LossAnchorGridShapelyRTREE:
    
    def __init__(self, num_classes=2,res=(10, 10), doprint=False, rotations=[0,30,60,90], aspect_ratios=[1,2,3,1/2,1/3], scale_k=20):
        """
        Paramters
        ---------
        num_classes (int): default=2
            num_classes is the number of different classes each anchor box
            can associate with (e.g. occupied, empty) 
            NOTICE: background will be included regardless of num_classes
        
        res (M, N): default=(10, 10)
            res is the resolution of the anchors, the init will create 
            an anchor grid of M rows x N cols
        
        """
        
        
        
        

        

        self.anchors = []        
        self.num_classes = num_classes
        self.res = res
        self.loss_f = FocalLoss(num_classes)
        
        self.num_anchors = res[1] * res[0]  * len(rotations) * len(aspect_ratios)  
        
        self.a_ic = torch.zeros([self.num_anchors, 5], dtype=torch.double)
        w, h = scale_k/res[0], scale_k/res[1]
        
#         mask_x = 20
#         mask_y = 20     
#         self.masks = np.array((res[0], res[1], mask_x * mask_y))
#         mask_shape = (res[0] * mask_x, res[1] * mask_y)
        
        anchorId = 0
        for xLeft in np.arange(res[1]):
            xRight = xLeft + 1
            for yBottom in np.arange(res[0]):
                yTop = yBottom + 1
                
                cx = (xLeft + .5)/res[1]
                cy = (yBottom + .5)/res[0]
             
                
                
                poly = Polygon([(xLeft/res[1],yBottom/res[0]),(xLeft/res[1],yTop/res[0])
                                ,(xRight/res[1],yTop/res[0]),(xRight/res[1],yBottom/res[0])])
                
                for angle in rotations:
                    polyRotated = rotate(poly,angle)
                    for aspect_ratio in aspect_ratios:
                        self.a_ic[anchorId] = torch.tensor([cx, cy, w*aspect_ratio**0.5, h*aspect_ratio**-0.5, angle])
                                
                        polyStretched = scale(polyRotated,xfact=aspect_ratio**0.5,yfact=aspect_ratio**-0.5)
                        polyStretched.id = anchorId
                        anchorId += 1
                        self.anchors.append(polyStretched)
        
        
        self.shapleyRTree = STRtree(self.anchors)
        print(len(self.anchors), self.num_anchors)
        self.threshold = .5 
        
    #    
    def getAnchorTensor(self):
        return self.a_ic
            
    def getIntersectingAnchors(self,boundingPoly, doprint=False):
        """
        returns all the intersecting anchors with the given polygon
        
        Parameters
        ---------
        boundingPoly: shapely Polygon
        """
        return self.shapleyRTree.query(boundingPoly)
    
    
    def getOverlapIou(self, boundingPolys, doprint=False):
        """
        returns the iou scores of anchors vs boundingPolys

        Parameters
        ----------
        boundingPolys: a list of shapely Polygons
        
        Returns
        -------
        overlaps: torch.Tensor((M,N), dtype=double)
            A torch array of size M (# of anchors) x N (# of boundingPolys)
            This array represents the IoU scores of each anchor box versus
            each bounding polygon object
        """
        
        
        
        overlaps = torch.zeros([self.num_anchors,len(boundingPolys)], dtype=torch.double)
#         fig, ax = plt.subplots(1,1, figsize=(9,9))

        plt_dict = defaultdict()

        for j, boundPoly in enumerate(boundingPolys):
            for anchorPoly in self.getIntersectingAnchors(boundPoly):
#                 draw_poly(ax, anchorPoly, c='green')
#                 draw_poly(ax, boundPoly, c='red')
                plt_dict[anchorPoly.id] = anchorPoly 
    
                inter = anchorPoly.intersection(boundPoly).area
                iou = inter / (anchorPoly.area + boundPoly.area - inter)
                overlaps[anchorPoly.id, j] = iou                
#         plt.show()
        return overlaps
    
    def getLoss(self, b_c, b_bb, bbox, clas, doprint=False):
        """
        Parameters
        ----------
        b_c: tensor(Num of Anchor boxes x num_classes + 1)
            predicted classifications
        b_bb: tensor(Num of Anchor boxes x 5)
            predicted bounding boxes
        clas: tensor(Nx1)
            a tensor of classifications for ground truth boxes
        bbox: tensor(Nx5)
            tensor of bounding boxes for the ground truth
        """
        
        print(b_bb.cpu().numpy())
        
        boundingPolys = create_polygon_list(b_bb.cpu().numpy())
        print(len(boundingPolys))
        '''
        for i in range(0, len(b_bb_list),5):
            print(b_bb_list[i],b_bb_list[i+1],b_bb_list[i+2],b_bb_list[i+3],b_bb_list[i+4])
            boundingPolys.append(create_polygon(b_bb_list[i],b_bb_list[i+1],b_bb_list[i+2],b_bb_list[i+3],b_bb_list[i+4]))
            
        print(b_bb.cpu().numpy())
        '''
        overlaps = self.getOverlapIou(boundingPolys, doprint)
        
        
        prior_overlap, prior_idx = overlaps.max(0) # for each ground truth object, find anchor box that has max overlap (highest jaccard index) with it
        gt_overlap, gt_idx = overlaps.max(1) # for each anchor box, find max jaccard index and which obj category this corresponds to
        gt_overlap[prior_idx] = 1.99 # set jaccard index of matched anchor boxes to 1.99
        for i,o in enumerate(prior_idx): gt_idx[o] = i # redefine the 'matched' indices in gt_idx to prior_idx indices. take a look below at gt_overlap and gt_idx values where gt_overlap = 1.99
        if doprint: print(gt_overlap, gt_idx)
        
        gt_clas = clas[gt_idx] # define the gt category for each matched anchor box
        pos = gt_overlap > self.threshold # bool array for anchor box jaccards > threshold value, i've changed from 0.3 to 0.2 to illustrate how gt_overlap[12] (==0.2598) is counted as 'pos'
        pos_idx = torch.nonzero(pos)[:,0]
        gt_clas[1-pos] = self.num_classes # setting class category to background
        gt_bbox = bbox[gt_idx] # define the gt_bboxes for each matched anchor box
        if doprint: print(gt_clas, gt_bbox)
            
            
        
        t = gt_bbox[pos_idx]
        p = self.a_ic[pos_idx]
        
        
        loc_loss =  torch.sum((t[:, 0] - p[:, 0]).abs()/p[:, 2])
        loc_loss += torch.sum((t[:, 1] - p[:, 1]).abs()/p[:, 3])
        loc_loss += torch.sum(torch.log(t[:, 2]/p[:, 2]))
        loc_loss += torch.sum(torch.log(t[:, 3]/p[:, 3]))
        loc_loss += torch.sum(torch.tan(t[:, 4] - p[:, 4]).abs())
        
#         ((self.a_ic[pos_idx][:4] - gt_bbox[pos_idx][:4]
#                     + torch.tan(self.a_ic[pos_idx][4] - gt_bbox[pos_idx][4]).abs()).mean() # calc regression loss (L1 = absolute mean distance between each predicted bbox value and gt value)
        
    
        print(b_c, gt_clas)
        clas_loss  = self.loss_f(b_c, gt_clas) # calc classification loss (binary cross entropy or focal loss)
        
        return loc_loss, clas_loss
    
    
    def ssdLoss(self, pred, targ, doprint=False):
        total_class_loss, total_loc_loss = 0., 0.
        for b_c, b_bb, bbox, clas in zip(*pred, *targ):
            loc_loss, clas_loss = self.getLoss()
            total_loc_loss += loc_loss
            total_class_loss += clas_loss
        
        return total_loc_loss + total_class_loss
        


In [13]:

bbox = torch.tensor([[.5,.2,.1,.1,15], [.3,.4,.3,.4,30], [.6,.6,.2,.2,0], [.3,.7,.1,.2,10]], dtype=torch.double)
clas = torch.tensor([1,0,0,1])

nn.SmoothL1Loss()

res = (600, 350)
bboxCount = res[0] * res[1] * 20

b_c_list = np.zeros((bboxCount, 3))
for i in np.arange(bboxCount):
    b_c_list[i, :] = np.random.dirichlet(np.ones(3),size=1)
    
    

b_c = torch.tensor(b_c_list)


b_bb = torch.tensor([[.5,.2,.1,.1,15], [.3,.4,.3,.4,30], [.6,.6,.2,.2,0], [.3,.7,.1,.2,10]], dtype=torch.double)


def testSpeed(b_c, b_bb, bbox, clas, res):
    testAnchor = LossAnchorGridShapelyRTREE(2, res)
    iouScore = testAnchor.getLoss(b_c, b_bb, bbox, clas,  False)
    testAnchor.getAnchorTensor()
    return iouScore

iouScore = testSpeed(b_c, b_bb, bbox, clas, res)
print(iouScore)
%timeit testSpeed(b_c, b_bb, bbox, clas, res)

4200000 4200000
[[ 0.5  0.2  0.1  0.1 15. ]
 [ 0.3  0.4  0.3  0.4 30. ]
 [ 0.6  0.6  0.2  0.2  0. ]
 [ 0.3  0.7  0.1  0.2 10. ]]
4
tensor([[0.5218, 0.2097, 0.2685],
        [0.4698, 0.3132, 0.2170],
        [0.6244, 0.3472, 0.0284],
        ...,
        [0.3192, 0.1387, 0.5421],
        [0.6196, 0.3575, 0.0229],
        [0.0556, 0.7900, 0.1543]], dtype=torch.float64) tensor([2, 2, 2,  ..., 2, 2, 2])
(tensor(37.5298, dtype=torch.float64), tensor(975691.4146, dtype=torch.float64))
4200000 4200000
[[ 0.5  0.2  0.1  0.1 15. ]
 [ 0.3  0.4  0.3  0.4 30. ]
 [ 0.6  0.6  0.2  0.2  0. ]
 [ 0.3  0.7  0.1  0.2 10. ]]
4
tensor([[0.5218, 0.2097, 0.2685],
        [0.4698, 0.3132, 0.2170],
        [0.6244, 0.3472, 0.0284],
        ...,
        [0.3192, 0.1387, 0.5421],
        [0.6196, 0.3575, 0.0229],
        [0.0556, 0.7900, 0.1543]], dtype=torch.float64) tensor([2, 2, 2,  ..., 2, 2, 2])
4200000 4200000
[[ 0.5  0.2  0.1  0.1 15. ]
 [ 0.3  0.4  0.3  0.4 30. ]
 [ 0.6  0.6  0.2  0.2  0. ]
 [ 0.3  0.7  